Load the __importnb__ to import notebooks.

In [1]:
    if __name__ == '__main__':
        %reload_ext importnb
    else: 
        foo = 42

In [2]:
    import readme
    assert readme.__file__.endswith('ipynb')
    assert readme.foo is 42

In [4]:
    if __name__ == '__main__':
        !jupyter nbconvert --to markdown readme.ipynb

[NbConvertApp] Converting notebook readme.ipynb to markdown
[NbConvertApp] Writing 356 bytes to readme.md
